In [1]:
import numpy as np
from scipy.optimize import fminbound


def bellman_operator(w, grid, beta, u, f, shocks, c_func, Tw=None):
    """
    The approximate Bellman operator, which computes and returns the
    updated value function Tw on the grid points.  An array to store
    the new set of values Tw is optionally supplied (to avoid having to
    allocate new arrays at each iteration).  If supplied, any existing data in 
    Tw will be overwritten.

    Parameters
    ----------
    w : array_like(float, ndim=1)
        The value of the input function on different grid points
    grid : array_like(float, ndim=1)
        The set of grid points
    beta : scalar
        The discount factor
    u : function
        The utility function
    f : function
        The production function
    shocks : numpy array
        An array of draws from the shock, for Monte Carlo integration (to
        compute expectations).
    Tw : array_like(float, ndim=1) optional (default=None)
        Array to write output values to

    """
    # === Apply linear interpolation to w === #
    w_func = lambda x: np.interp(x, grid, w)
    
    if Tw is None:
        Tw = np.empty_like(w)

    # == set Tw[i] = max_c { u(c) + beta E w(f(y  - c) z)} == #
    for i, y in enumerate(grid):
        def objective(c):
            return - u(c_func(y)) - beta * np.mean(w_func(f(y - c_func(y)) * shocks))
        c_star = fminbound(objective, 1e-10, y)
        Tw[i] = - objective(c_star)


    return Tw


In [2]:
from quantecon import compute_fixed_point

grid_max = 4
grid_size = 200
shock_size = 250

grid = np.linspace(1e-5, grid_max, grid_size)
shocks = np.exp(0 + 0.1 * np.random.randn(shock_size))

Tw = np.empty(len(grid))
initial_w = 5* np.log(grid)

c_func_1 = lambda y: 0.95*y
c_func_2 = lambda y: scipy.stats.lognorm.cdf(y,0.5)*y

T = lambda w: bellman_operator(w,
                               grid,
                               0.96,
                               np.log,
                               lambda k: k**0.4,
                               shocks,
                               c_func_1)

v_star_approx = compute_fixed_point(T, initial_w,
                                    error_tol=1e-5,
                                    max_iter=500,
                                    verbose=2,
                                    print_skip=10,
                                    method='iteration')


Iteration    Distance       Elapsed (seconds)
---------------------------------------------
10           1.285e+00      1.999e+00         
20           7.819e-01      3.812e+00         
30           5.103e-01      5.822e+00         
40           3.380e-01      7.846e+00         


KeyboardInterrupt: 

In [ ]:
import scipy

T2 = lambda w: bellman_operator(w,
                               grid,
                               0.96,
                               np.log,
                               lambda k: k**0.4,
                               shocks,
                               c_func_2)

v_star_approx_2 = compute_fixed_point(T2, initial_w,
                                     error_tol=1e-5,
                                     max_iter=500,
                                     verbose=2,
                                     print_skip=10,
                                     method='iteration')

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(9,5))
ax.set_ylim(-150, -20)
ax.plot(grid, v_star_approx, lw=2, color='b', label='0.95y')
ax.plot(grid, v_star_approx_2, lw=2, color='r', label='H(y)y')
plt.show()